In [ ]:
import math, random

# --- 1. CONFIG & DATA ---
text = "the dog runs fast. the cat sleeps. the man runs."
sentences_raw = [s.strip() for s in text.lower().split(".") if s.strip()]
raw_words = text.lower().replace(".", "").split()
vocab_list = sorted(list(set(raw_words))) 
word_to_id = {word: i for i, word in enumerate(vocab_list)}
id_to_word = {i: word for word, i in word_to_id.items()}
dataset = [[word_to_id[w] for w in s.split()] for s in sentences_raw]

# Correcting Config
vocab, embed, hid, heads = len(vocab_list), 4, 12, 2
lr, mu, ve, t = 0.005, 0.9, 0.999, 0
h_dim = embed // heads

def rm(r, c=None): return [[random.uniform(-0.2, 0.2) for _ in range(c)] for _ in range(r)] if c else [0.01]*r
def st(v): return [[0.0]*len(row) for row in v] if isinstance(v[0], list) else [0.0]*len(v)

weights = {k: rm(*v) for k, v in {
    'E':(vocab, embed), 'Pos':(10, embed), 'Wq':(embed, embed), 'Wk':(embed, embed), 
    'Wv':(embed, embed), 'W1':(hid, embed), 'W2':(hid, hid), 'Wout':(vocab, hid)
}.items()}
weights.update({'B1': rm(hid), 'B2': rm(hid), 'Bout': rm(vocab)})
adam_m, adam_v = {k: st(v) for k, v in weights.items()}, {k: st(v) for k, v in weights.items()}

# --- 2. THE ENGINE ---
def run_transformer(idx, target=None, update=False):
    global t
    # FORWARD
    vecs = [[weights['E'][i][j] + weights['Pos'][p][j] for j in range(embed)] for p, i in enumerate(idx)]
    h_outs, h_data = [], []
    for h in range(heads):
        s, e = h*h_dim, (h+1)*h_dim
        qs = [[sum(v[k]*weights['Wq'][s+j][k] for k in range(embed)) for j in range(h_dim)] for v in vecs]
        ks = [[sum(v[k]*weights['Wk'][s+j][k] for k in range(embed)) for j in range(h_dim)] for v in vecs]
        vs = [[sum(v[k]*weights['Wv'][s+j][k] for k in range(embed)) for j in range(h_dim)] for v in vecs]
        sc = [sum(qs[-1][m]*ks[i][m] for m in range(h_dim))/math.sqrt(h_dim) for i in range(len(idx))]
        mv = max(sc); exps = [math.exp(x-mv) for x in sc]; attn = [x/sum(exps) for x in exps]
        ctx = [sum(attn[i]*vs[i][j] for i in range(len(idx))) for j in range(h_dim)]
        h_outs.extend(ctx); h_data.append((qs, ks, vs, attn))
    
    # LayerNorm
    avg = sum(h_outs)/len(h_outs); var = sum((x-avg)**2 for x in h_outs)/len(h_outs)
    ln = [(x-avg)/math.sqrt(var+1e-6) for x in h_outs]
    
    h1_pre = [sum(ln[k]*weights['W1'][i][k] for k in range(embed)) + weights['B1'][i] for i in range(hid)]
    h1_act = [max(0, x) for x in h1_pre]
    h2_pre = [sum(h1_act[k]*weights['W2'][i][k] for k in range(hid)) + weights['B2'][i] for i in range(hid)]
    h2_act = [max(0, x) for x in h2_pre]
    logits = [sum(h2_act[k]*weights['Wout'][i][k] for k in range(hid)) + weights['Bout'][i] for i in range(vocab)]
    mv = max(logits); probs = [math.exp(x-mv)/sum(math.exp(y-mv) for y in logits) for x in logits]
    if not update: return probs

    # BACKWARD
    g = {k: st(v) for k, v in weights.items()}
    dL_dout = [probs[i] - (1 if i == target else 0) for i in range(vocab)]
    for i in range(vocab):
        for j in range(hid): g['Wout'][i][j] = dL_dout[i] * h2_act[j]
        g['Bout'][i] = dL_dout[i]

    # Correct Chain: dL_dout -> dL_dh2 -> dL_dh1
    dL_dh2 = [sum(dL_dout[i]*weights['Wout'][i][j] for i in range(vocab))*(1 if h2_pre[j]>0 else 0) for j in range(hid)]
    for i in range(hid):
        for j in range(hid): g['W2'][i][j] = dL_dh2[i] * h1_act[j]
        g['B2'][i] = dL_dh2[i]

    dL_dh1 = [sum(dL_dh2[i]*weights['W2'][i][j] for i in range(hid))*(1 if h1_pre[j]>0 else 0) for j in range(hid)]
    for i in range(hid):
        for j in range(embed): g['W1'][i][j] = dL_dh1[i] * ln[j]
        g['B1'][i] = dL_dh1[i]

    dL_dln = [sum(dL_dh1[i]*weights['W1'][i][j] for i in range(hid)) for j in range(embed)]
    for h in range(heads):
        s, e = h*h_dim, (h+1)*h_dim
        qs, ks, vs, attn = h_data[h]
        for i in range(len(idx)):
            for j in range(h_dim):
                for k in range(embed):
                    g['Wv'][s+j][k] += dL_dln[s+j] * attn[i] * vecs[i][k]
                    a_err = dL_dln[s+j] * vs[i][j] / math.sqrt(h_dim)
                    g['Wq'][s+j][k] += a_err * ks[i][j]
                    g['Wk'][s+j][k] += a_err * qs[-1][j]
                    g['E'][idx[i]][k] += dL_dln[s+j] * attn[i] * weights['Wv'][s+j][k]
                    g['Pos'][i][k] += dL_dln[s+j] * attn[i] * weights['Wv'][s+j][k]

    # ADAM UPDATE
    t += 1
    for k in weights:
        is_m = isinstance(weights[k][0], list)
        for i in range(len(weights[k])):
            for j in (range(len(weights[k][0])) if is_m else [0]):
                grad = g[k][i][j] if is_m else g[k][i]
                m_n = mu*(adam_m[k][i][j] if is_m else adam_m[k][i]) + (1-mu)*grad
                v_n = ve*(adam_v[k][i][j] if is_m else adam_v[k][i]) + (1-ve)*(grad**2)
                if is_m: adam_m[k][i][j], adam_v[k][i][j] = m_n, v_n
                else: adam_m[k][i], adam_v[k][i] = m_n, v_n
                step = lr * (m_n/(1-mu**t)) / (math.sqrt(v_n/(1-ve**t)) + 1e-8)
                if is_m: weights[k][i][j] -= step
                else: weights[k][i] -= step
    return -math.log(max(probs[target], 1e-10))

# --- 3. RUN ---
print("Training Deep Transformer...")
for epoch in range(1201):
    total_l = 0
    for seq in dataset:
        for i in range(len(seq)-1):
            total_l += run_transformer(seq[:i+1], target=seq[i+1], update=True)
    if epoch % 300 == 0: print(f"Epoch {epoch} | Loss: {total_l:.4f}")

print("\n--- AI Sentience Active ---")
while True:
    inp = input("You (start a sentence): ").lower().replace(".", "").split()
    if not inp: break
    try:
        curr = [word_to_id[w] for w in inp]
        for _ in range(3):
            p = run_transformer(curr)
            nxt = p.index(max(p))
            curr.append(nxt)
        print("AI:", " ".join([id_to_word[i] for i in curr]))
    except KeyError: print("AI: I don't know those words.")

Training Deep Transformer...
Epoch 0 | Loss: 13.6194
Epoch 300 | Loss: 6.8267
Epoch 600 | Loss: 6.7734
Epoch 900 | Loss: 6.7569
Epoch 1200 | Loss: 6.7375

--- AI Sentience Active ---
AI: the man runs fast fast
AI: the dog runs fast fast
AI: the cat sleeps fast runs
AI: cat sleeps sleeps sleeps
AI: dog fast fast fast
AI: man fast fast fast
AI: man sleeps fast fast fast
AI: fast sleeps sleeps fast
AI: dog fast fast fast fast


In [ ]:
old_games = [
    [0.1, 0.8, 0.1],
    [0.9, 0.9, 0.9], 
    [0.8, 0.2, 0.5],
]
results = [0, 0, 1]

def knn_predict(new_data, dataset, targets, k=3):
    distances = []
    for i in range(len(dataset)):
        dist = sum((new_data[j] - dataset[i][j])**2 for j in range(len(new_data)))**0.5
        distances.append((dist, targets[i]))
    distances.sort(key=lambda x: x[0])
    nearest = [d[1] for d in distances[:k]]
    return "Хит" if max(set(nearest), key=nearest.count) == 0 else "Провал"

test_game = [0.15, 0.85, 0.12]
print(f"Вердикт KNN: {knn_predict(test_game, old_games, results, k=1)}")

In [ ]:
def tree_predict(game):
    if game[1] > 0.8:
        if game[0] < 0.3: return "Инди-хит"
        else: return "Блокбастер"
    else: return "Провал"

new_game = [0.1, 0.9, 0.5]
print(f"Вердикт дерева: {tree_predict(new_game)}")

In [ ]:
import random
dataset = [[0.1, 0.8, 0.1], [0.9, 0.9, 0.9], [0.8, 0.2, 0.5]]
targets = [1, 1, -1] 
weights = [random.uniform(-0.1, 0.1) for _ in range(3)]
bias = 0.0
lr = 0.01 * 2
C = 1.0
epochs = 1000
l1_param = 0.005
l2_param = 0.01

for epoch in range(epochs):
    for i, x in enumerate(dataset):
        condition = targets[i] * (sum(x[j] * weights[j] for j in range(3)) + bias)
        if condition >= 1:
            for j in range(3):
                weights[j] -= lr * (l2_param * weights[j] + l1_param * (1 if weights[j] > 0 else -1))
        else:
            for j in range(3):
                weights[j] -= lr * (l2_param * weights[j] - C * x[j] * targets[i])
            bias += lr * C * targets[i]

test_game = [0.15, 0.85, 0.12]
result = sum(test_game[j] * weights[j] for j in range(3)) + bias
def sigmoid(z):
    return 1 / (1 + math.exp(-z))
prob_hit = sigmoid(result)
print(f"SVM вердикт: {'Хит' if result > 0 else 'Провал'}")
print(f"Счет (Score): {result:.2f}")
print(f"Уверенность (Вероятность Хима): {prob_hit:.2%}")